# Задание 5

## 1. ContextTimer(0.3 балла)
Напишите менеджер контекста, который позволит засекать время выполнения блока кода с помощью конструкции with и выводить это время на экран по выходу из блока. Пример использования:

<code>
with Timer ():
    do_some_long_stuff()
</code>

Рекомендации: используйте стандартную библиотеку для работы со временем.

Протестируйте себя.

In [5]:
from datetime import datetime
import time


class Timer:
    def __enter__(self):
        self.start_time = datetime.now()
        
    def __exit__(self, type, value, traceback):
        print("Elapsed time: {}".format(datetime.now() - self.start_time))

In [6]:
with Timer():
    time.sleep(1)
    raise Exception("SOS")
    time.sleep(10000)

Elapsed time: 0:00:01.001506


Exception: SOS

In [7]:
with Timer():   
    %time 2 ** 999999999

CPU times: user 8 µs, sys: 18 µs, total: 26 µs
Wall time: 66 µs
Compiler : 8.92 s
Elapsed time: 0:00:39.530424


In [8]:
with Timer():   
    2 ** 999999999

Elapsed time: 0:00:00.000161


## 2.Transaction (0.4 балла)
Вам необходимо написать менеджер контекстов, который позволит безопасно работать с транзакциями. Напишите класс Storage, в котором будут храниться какие-то данные в виде словаря. Эти данные должны быть закрытыми и их можно только читать через операцию []. У этого класса должен быть метод edit, который возвращает менеджер контекста, позволяющий редактировать исходный объект (опять же через []). При этом результаты редактирования записываются в исходный объект только если весь блок выполнился успешно. Пример использования:

<code>
s = Storage()
with s.edit() as se :
    se["a"] = 1
    may_be_an_exception_here()
</code>

Протестируйте себя.

In [80]:
import re

class Storage:
    def __init__(self):
        self.__data = {}
    
    def __repr__(self):
        return self.__data.__repr__()
    
    def __str__(self):
        return self.__data.__str__()
    
    def __getitem__(self, name):
        return self.__data[name]
    
    def edit(self):
        return StorageManager(self)
    

class StorageManager:
    def __init__(self, storage):
        self.__storage = storage
        self.__changes = {}
    
    def __setitem__(self, name, value):
        self.__changes[name] = value
    
    def __getitem__(self, name):
        return self.__changes[name]
    
    def __enter__(self):
        return self
    
    def __exit__(self, type, value, traceback):
        if traceback is None:
            self.__storage._Storage__data.update(self.__changes)

In [88]:
s = Storage()

with s.edit() as sm:
    sm["number"] = 10
    sm["name"] = "First name"

In [89]:
try:
    with s.edit() as sm:
        sm["number"] = 100
        raise Exception("SOS")
except:
    pass

s

{'number': 10, 'name': 'First name'}

In [92]:
try:
    with s.edit() as sm:
        sm["number"] = 100
        sm["surname"] = "Second name"
except:
    pass

s

{'number': 100, 'name': 'First name', 'surname': 'Second name'}

## Phone numbers (0.3)
Напишите регулярное выражение для распознавания телефонных номеров и протестируйте себя. Номера, которые должны распознаваться:
* 3-22-46
* +71239513749
* 71239513749
* 1239513749
* +7(123)-951-37-49
* +7(123)9513749
* +7-123-9513749
* +7-123-951-37-49

* 7(123)-951-37-49
* 7(123)9513749
* 7-123-9513749
* 7-123-951-37-49

* (123)-951-37-49
* (123)9513749
* 123-9513749
* 123-951-37-49

In [96]:
import re

phone_dataset = [
    "3-22-46",
    "+71239513749",
    "71239513749",
    "1239513749",
    "+7(123)-951-37-49",
    "+7(123)9513749",
    "+7-123-9513749",
    "+7-123-951-37-49",
    "7(123)-951-37-49",
    "7(123)9513749",
    "7-123-9513749",
    "7-123-951-37-49",
    "(123)-951-37-49",
    "(123)9513749",
    "123-9513749",
    "123-951-37-49",
]

text = " some text".join(phone_dataset)
text

'3-22-46 some text+71239513749 some text71239513749 some text1239513749 some text+7(123)-951-37-49 some text+7(123)9513749 some text+7-123-9513749 some text+7-123-951-37-49 some text7(123)-951-37-49 some text7(123)9513749 some text7-123-9513749 some text7-123-951-37-49 some text(123)-951-37-49 some text(123)9513749 some text123-9513749 some text123-951-37-49'

In [125]:
phone_regex = re.compile(
    r"([+]?(7)?[(-]?(\d{3})?[)]?[-]?(\d{3})[-]?(\d{2})[-]?(\d{2}))|((\d)[-]?(\d{2})[-]?(\d{2}))"
)

In [136]:
phone_regex.findall(text)

[('', '', '', '', '', '', '3-22-46', '3', '22', '46'),
 ('+71239513749', '7', '123', '951', '37', '49', '', '', '', ''),
 ('71239513749', '7', '123', '951', '37', '49', '', '', '', ''),
 ('1239513749', '', '123', '951', '37', '49', '', '', '', ''),
 ('+7(123)-951-37-49', '7', '123', '951', '37', '49', '', '', '', ''),
 ('+7(123)9513749', '7', '123', '951', '37', '49', '', '', '', ''),
 ('+7-123-9513749', '7', '123', '951', '37', '49', '', '', '', ''),
 ('+7-123-951-37-49', '7', '123', '951', '37', '49', '', '', '', ''),
 ('7(123)-951-37-49', '7', '123', '951', '37', '49', '', '', '', ''),
 ('7(123)9513749', '7', '123', '951', '37', '49', '', '', '', ''),
 ('7-123-9513749', '7', '123', '951', '37', '49', '', '', '', ''),
 ('7-123-951-37-49', '7', '123', '951', '37', '49', '', '', '', ''),
 ('(123)-951-37-49', '', '123', '951', '37', '49', '', '', '', ''),
 ('(123)9513749', '', '123', '951', '37', '49', '', '', '', ''),
 ('123-9513749', '', '123', '951', '37', '49', '', '', '', ''),
 ('1

In [137]:
def normalize_phone(groups):
    groups = list(groups)
    groups[1] = '+7'
    if groups[0] != '':
        return groups[1] + groups[2] + groups[3] + groups[4]
    else:
        return groups[7] + groups[8] + groups[9]

In [138]:
list(map(normalize_phone, phone_regex.findall(text)))

['32246',
 '+712395137',
 '+712395137',
 '+712395137',
 '+712395137',
 '+712395137',
 '+712395137',
 '+712395137',
 '+712395137',
 '+712395137',
 '+712395137',
 '+712395137',
 '+712395137',
 '+712395137',
 '+712395137',
 '+712395137']